In [1]:
from constants import *
from utils import *

In [2]:
encode_features()

Working with MLFlow DB: /home/Databases/Lead_scoring_mlflow_production.db
MLFLOW DB Already Exists at /home/Databases/Lead_scoring_mlflow_production.db
/home/assignment/02_training_pipeline/scripts
Reading the cleaned data from the database /home/Databases/lead_scoring_data_cleaning.db
Reading Complete!


In [3]:
get_trained_model()

Reading the features and target from the database /home/Databases/lead_scoring_data_cleaning.db


Registered model 'LightGBM' already exists. Creating a new version of this model...
2024/02/07 03:25:26 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: LightGBM, version 3
Created version '3' of model 'LightGBM'.
/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Inside MLflow Run with id 79537cb55cbc40e0b15534bdd2cc2c56


In [2]:

import pandas as pd
import numpy as np

import sqlite3
from sqlite3 import Error

import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#from Lead_scoring_training_pipeline.constants import *

from constants import *

In [ ]:
db_file_path = DB_PATH + DB_FILE_NAME    
db_conn = None    

In [ ]:
print(f"Reading the cleaned data from the database {db_file_path}")
db_conn = sqlite3.connect(db_file_path)
cleaned_data = pd.read_sql('SELECT * FROM model_input', con=db_conn)
db_conn.close()
db_conn = None


In [ ]:
data = cleaned_data.copy()

In [ ]:
data.head()

In [ ]:
ONE_HOT_ENCODED_FEATURES

In [ ]:
FEATURES_TO_ENCODE

In [ ]:
for f in FEATURES_TO_ENCODE:
    if(f in data.columns):
        encoded = pd.get_dummies(data[f])
        encoded = encoded.add_prefix(f + '_')
        data = pd.concat([data, encoded], axis=1)
    else:
        print('Feature not found')


In [ ]:
print(data.columns)

In [ ]:
data.drop(columns=FEATURES_TO_ENCODE, inplace = True)

In [ ]:
data[ONE_HOT_ENCODED_FEATURES]

In [ ]:
data_subset = data[ONE_HOT_ENCODED_FEATURES]

In [ ]:
target_column = data_subset.pop('app_complete_flag')
data_subset

In [ ]:
target_column.head()

In [ ]:
data_subset.head()

In [ ]:
############# -------------------------------

In [3]:

db_file_path = DB_PATH + DB_FILE_NAME
db_conn = None

In [4]:
print(f"Reading the training_features and training_target from the database {db_file_path}")
db_conn = sqlite3.connect(db_file_path)

features = pd.read_sql('SELECT * FROM training_features', con=db_conn)
target_col = pd.read_sql('SELECT * FROM training_target', con=db_conn)

db_conn.close()
db_conn = None

X_train, X_test, y_train, y_test = train_test_split(features, target_col, test_size = 0.3, random_state=42)

#mlflow.set_tracking_uri(TRACKING_URI)

Reading the training_features and training_target from the database /home/airflow/dags/Lead_scoring_data_pipeline/lead_scoring_data_cleaning.db


In [ ]:
features.head()

In [5]:
mlflow.set_tracking_uri(TRACKING_URI)

In [6]:
lightgbm_model = lgb.LGBMClassifier()
lightgbm_model.set_params(**model_config)                   # model_config from Constants.py

LGBMClassifier(device='gpu', random_state=42)

In [8]:
with mlflow.start_run(run_name="run_LGBMClassifier_Without_HPTune") as run:
    lightgbm_model.fit(X_train, y_train)

    y_pred = lightgbm_model.predict(X_test)

    acc=accuracy_score(y_pred, y_test)
    mlflow.log_metric("test_accuracy", acc)

    auc = roc_auc_score(y_pred, y_test)
    mlflow.log_metric("auc", auc)

/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [10]:
model_info = mlflow.sklearn.log_model(sk_model=lightgbm_model, artifact_path='models', registered_model_name='LightGBM')

Registered model 'LightGBM' already exists. Creating a new version of this model...
2024/02/06 18:06:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: LightGBM, version 4
Created version '4' of model 'LightGBM'.


In [9]:
mlflow.sklearn.save_model(lightgbm_model, registered_model_name='LightGBM')

TypeError: save_model() got an unexpected keyword argument 'registered_model_name'

In [14]:
mlflow.sklearn.save_model(lightgbm_model, model_info.model_uri)

In [12]:
model_info.model_uri

'runs:/a39d84fd474843e98fd885881e692175/models'

In [16]:
mlflow.sklearn.save_model(sk_model=lightgbm_model, path=model_info.model_uri)

MlflowException: Path 'runs:/a39d84fd474843e98fd885881e692175/models' already exists and is not empty

In [17]:
mlflow.end_run()

NameError: name 'datetime' is not defined

In [20]:
k = pd.DataFrame(columns=['A'])
k['A'] = (1,2,3,4,5,6,7,8)

In [25]:
len(k[k.A == 1])

1